## **DATA MINING PROJECT**

**Aybike Battal - 150200343**


**İlayda Kara - 150220747**

In [1]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, LabelEncoder
from sklearn.cluster import KMeans
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.model_selection import train_test_split

### **Data Preparation**

In [2]:
# Read files and convert them to DataFrames
product_catalog = pd.read_csv("product_catalog.csv")
product_category_map = pd.read_csv("product_category_map.csv")
transactions = pd.read_csv("transactions.csv")

Data Overview

In [3]:
# Information about Product Catalog Dataset
print("Info for product_catalog df:")
print(product_catalog.info())
print()
print("Number of Duplicate Rows:")
print(product_catalog.duplicated().sum())
print()
print("Missing Value Number for Each Attribute:")
print(product_catalog.isnull().sum())
print()
print("First three rows of df:")
display(product_catalog.head(3))

Info for product_catalog df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32776 entries, 0 to 32775
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   product_id       32776 non-null  int64 
 1   manufacturer_id  32776 non-null  int64 
 2   attribute_1      32776 non-null  int64 
 3   attribute_2      32776 non-null  int64 
 4   attribute_3      32776 non-null  int64 
 5   attribute_4      32776 non-null  int64 
 6   attribute_5      32776 non-null  int64 
 7   categories       25988 non-null  object
dtypes: int64(7), object(1)
memory usage: 2.0+ MB
None

Number of Duplicate Rows:
0

Missing Value Number for Each Attribute:
product_id            0
manufacturer_id       0
attribute_1           0
attribute_2           0
attribute_3           0
attribute_4           0
attribute_5           0
categories         6788
dtype: int64

First three rows of df:


,product_id,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]"
1,28640,1366,10,1,537,0,101,NaN
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3..."


In [4]:
# Information about Product Category Map Dataset
print("Info for product_category_map df:")
print(product_category_map.info())
print()
print("Missing Value Number for Each Attribute:")
print(product_category_map.isnull().sum())
print()
print("Number of Duplicate Rows:")
print(product_category_map.duplicated().sum())
print()
print("First three rows of df:")
display(product_category_map.head(3))

Info for product_category_map df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332 entries, 0 to 4331
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   category_id         4332 non-null   int64
 1   parent_category_id  4332 non-null   int64
dtypes: int64(2)
memory usage: 67.8 KB
None

Missing Value Number for Each Attribute:
category_id           0
parent_category_id    0
dtype: int64

Number of Duplicate Rows:
0

First three rows of df:


,category_id,parent_category_id
0,0,75
1,1,1499
2,2,1082


In [5]:
# Information about Transactions Dataset
print("Info for transactions df:")
print(transactions.info())
print()
print("Missing Value Number for Each Attribute:")
print(transactions.isnull().sum())
print()
print("First three rows of df:")
display(transactions.head(3))

Info for transactions df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071538 entries, 0 to 1071537
Data columns (total 4 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   customer_id    1071538 non-null  int64 
 1   product_id     1071538 non-null  int64 
 2   purchase_date  1071538 non-null  object
 3   quantity       1071538 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 32.7+ MB
None

Missing Value Number for Each Attribute:
customer_id      0
product_id       0
purchase_date    0
quantity         0
dtype: int64

First three rows of df:


,customer_id,product_id,purchase_date,quantity
0,38769,3477,2020-06-01,1
1,42535,30474,2020-06-01,1
2,42535,15833,2020-06-01,1


Handling Missing Data

In [6]:
# Fill any missing values in the 'categories' column with an empty list '[]' 
# to ensure there are no null values in the column
product_catalog['categories'] = product_catalog['categories'].fillna('[]')
# Print the count of missing values in each column 
print(product_catalog.isnull().sum())

product_id         0
manufacturer_id    0
attribute_1        0
attribute_2        0
attribute_3        0
attribute_4        0
attribute_5        0
categories         0
dtype: int64


Feature Engineering

In [7]:
# Convert string representation of lists in 'categories' column to actual Python lists
product_catalog['categories'] = product_catalog['categories'].apply(literal_eval)

# Encode categories using MultiLabelBinarizer to convert lists into binary format
mlb = MultiLabelBinarizer()
categories_encoded = mlb.fit_transform(product_catalog['categories'])

# Normalize numerical features
numerical_features = product_catalog[['attribute_1', 'attribute_2', 'attribute_3', 'attribute_4', 'attribute_5']]
scaler = StandardScaler()
numerical_features_normalized = scaler.fit_transform(numerical_features)

# Combine the normalized numerical features and encoded categories into a single feature set
features = np.hstack([numerical_features_normalized, categories_encoded])

In [8]:
# Apply KMeans clustering with 10 clusters and assign cluster labels to the data
kmeans = KMeans(n_clusters=10, random_state=42)
product_catalog['cluster'] = kmeans.fit_predict(features)

In [9]:
# Convert 'purchase_date' to datetime format
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])

In [10]:
# Add new feature called 'shopping tendency'

# Count the number of transactions per customer
transaction_counts = transactions.groupby('customer_id')['purchase_date'].count().reset_index(name='transaction_count')

# Discretize the transaction counts into categories from 1 to 10
max_transactions = transaction_counts['transaction_count'].max()
bins = np.linspace(0, max_transactions, 11)  # 10 bins
labels = range(1, 11)  # Categories 1 to 10
customer_categories = pd.cut(transaction_counts['transaction_count'], bins=bins, labels=labels, include_lowest=True)

# Add the discretized 'shopping_tendency' category to the transaction data
transaction_counts['shopping_tendency'] = customer_categories

# Merge transaction count and shopping tendency back into 'transactions' DataFrame
transactions = pd.merge(transactions, transaction_counts[['customer_id', 'shopping_tendency']], on='customer_id', how='left')

*Binning the transaction counts into categories: The number of transactions per customer is divided into 10 bins. Each bin represents a range of transaction counts, and each customer is placed into a category based on their transaction frequency.*

In [11]:
# Calculate the number of days since the last purchase for each customer-product pair

# Sort transactions by customer_id, product_id, and purchase_date
transactions = transactions.sort_values(by=['customer_id', 'product_id', 'purchase_date'])

# Calculate the previous purchase date for each customer-product pair
transactions['prev_purchase_date'] = transactions.groupby(['customer_id', 'product_id'])['purchase_date'].shift(1)

# Calculate the number of days since last purchase and fill missing values with 0
transactions['days_since_last_purchase'] = (transactions['purchase_date'] - transactions['prev_purchase_date']).dt.days.fillna(0).astype(int)

# Apply log transformation to 'days_since_last_purchase' for better scale
transactions['log_days_since_last_purchase'] = np.log1p(transactions['days_since_last_purchase'])

*The 'days since last purchase' feature is calculated to assess the likelihood of a customer repurchasing the same product within a specified time frame.*

*The 'log_days_since_last_purchase' feature is created by applying a log transformation to the 'days_since_last_purchase' in order to better handle large variations in the number of days. This transformation helps to reduce the impact of outliers and compresses the scale, making the data more suitable for modeling.*

In [12]:
# Calculate product popularity based on the total number of transactions per product
product_popularity = transactions.groupby('product_id')['customer_id'].count().reset_index(name='product_popularity')

# Merge product popularity into 'transactions' DataFrame
transactions = pd.merge(transactions, product_popularity[['product_id', 'product_popularity']], on='product_id', how='left')

*The product_popularity feature is calculated to see how frequently a product is purchased across all customers, helping to understand the overall demand for each product.*

In [13]:
# Calculate the quantity of the last purchase for each customer-product pair
transactions['last_quantity'] = transactions.groupby(['customer_id', 'product_id'])['quantity'].shift(1, fill_value=0).astype(int)

*The 'last_quantity' feature is calculated to analyze if the quantity purchased in the previous transaction influences the time taken for a customer to repurchase a product. For example, purchasing a larger quantity might indicate a longer wait before the next purchase.*

In [14]:
# Calculate days since the first purchase for each customer
first_purchase = transactions.groupby('customer_id')['purchase_date'].min().reset_index()
first_purchase.columns = ['customer_id', 'first_purchase_date']
first_purchase['days_since_first_purchase'] = (pd.to_datetime('today') - first_purchase['first_purchase_date']).dt.days

# Merge the 'days_since_first_purchase' feature into 'transactions' DataFrame
transactions = pd.merge(transactions, first_purchase[['customer_id', 'days_since_first_purchase']], on='customer_id', how='left')

*The 'days_since_first_purchase' feature is used to measure how long a customer has been using the platform. This can help in understanding customer behavior, as older customers may have different repurchase patterns than newer ones.*

In [15]:
# Calculate rolling average of quantity for each customer over the past 5 transactions
transactions['rolling_avg_quantity'] = transactions.groupby('customer_id')['quantity'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

# Calculate rolling mean of days between purchases for each customer
transactions['rolling_avg_days_between'] = transactions.groupby('customer_id')['days_since_last_purchase'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

*The rolling average features are calculated to analyze the customer's past behavior and identify trends in their purchasing patterns. By considering the last 5 transactions, we can observe how the quantity purchased and the time between purchases impact future buying behavior. This approach helps in predicting when the customer is likely to repurchase, as past purchasing trends often influence future buying decisions.*

In [16]:
# Calculate the total quantity purchased by each customer
total_quantity = transactions.groupby('customer_id')['quantity'].sum().reset_index(name='total_quantity')

# Calculate the average quantity purchased per transaction for each customer
average_quantity = transactions.groupby('customer_id')['quantity'].mean().reset_index(name='average_quantity')

# Merge these features into a single DataFrame for customer_features
customer_features = pd.merge(transaction_counts[['customer_id', 'transaction_count']], total_quantity, on='customer_id')
customer_features = pd.merge(customer_features, average_quantity, on='customer_id')

# Add preferred shopping time based on the mode of the day of week for each customer - Weekday or Weekend
transactions['day_of_week'] = transactions['purchase_date'].dt.dayofweek
transactions['is_weekend'] = transactions['day_of_week'].isin([5, 6])  # 5=Saturday, 6=Sunday
customer_mode_day = transactions.groupby('customer_id')['day_of_week'].agg(lambda x: x.mode()[0]).reset_index()
customer_mode_day.columns = ['customer_id', 'mode_day_of_week']
transactions = pd.merge(transactions, customer_mode_day, on='customer_id', how='left')
transactions['preferred_shopping_time'] = transactions['mode_day_of_week'].apply(lambda x: 'Weekend' if x in [5, 6] else 'Weekday')
transactions['preferred_shopping_time_encoded'] = transactions['preferred_shopping_time'].apply(lambda x: 1 if x == 'Weekend' else 0)

# Merge preferred shopping time encoded feature into the customer_features DataFrame
customer_features = pd.merge(customer_features, transactions[['customer_id', 'preferred_shopping_time_encoded']].drop_duplicates(), on='customer_id', how='left')

# Scale features to have a similar range and improve K-means performance
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features[['transaction_count', 
                                                         'total_quantity', 
                                                         'average_quantity', 
                                                         'preferred_shopping_time_encoded']])

*The features like total quantity, average quantity, and preferred shopping time are used to group customers with similar purchasing patterns for better analysis and predictions.*

In [17]:
# Apply KMeans clustering with 10 clusters and assign cluster labels to the data
kmeans = KMeans(n_clusters=10, random_state=42)
customer_features['cluster_customer'] = kmeans.fit_predict(scaled_features)

# Merge customer clusters back into the transactions DataFrame
transactions = pd.merge(transactions, customer_features[['customer_id', 'cluster_customer']], on='customer_id', how='left')

In [18]:
# Merge transaction data with product catalog to add cluster information
merged_df = pd.merge(
    transactions,
    product_catalog[['product_id', 'cluster']],  # Select columns for merging 
    on='product_id',
    how='left'  
)

# Sort by customer_id, cluster, and purchase_date to keep the correct order for calculations.
merged_df = merged_df.sort_values(by=['customer_id', 'cluster', 'purchase_date'])

# Calculate cumulative sum of quantities for each customer-cluster to track total quantity over time
merged_df['cumulative_quantity'] = merged_df.groupby(['customer_id', 'cluster'])['quantity'].cumsum()

# Calculate the previous transaction's cumulative quantity
merged_df['last_cumulative_quantity'] = merged_df.groupby(['customer_id', 'cluster'])['cumulative_quantity'].shift(1, fill_value=0).astype(int)

# Calculate the last quantity purchased by the same customer for the same product
merged_df['last_quantity'] = merged_df.groupby(['customer_id', 'product_id'])['quantity'].shift(1, fill_value=0).astype(int)

# Select columns for training the model
train_df = merged_df[['customer_id', 'product_id', 'cluster', 'last_quantity',
                       'log_days_since_last_purchase', 'days_since_last_purchase', 'purchase_date', 
                       'quantity', 'cumulative_quantity', 'last_cumulative_quantity', 
                       'shopping_tendency', 'cluster_customer', 'product_popularity', 
                       'days_since_first_purchase', 'rolling_avg_quantity', 'rolling_avg_days_between']]


*Transactions and product catalog dataframes are merged to find features for training the model. Additional features, such as previous cumulative quantity are added to track how many products a customer has purchased from the same product cluster. These features help represent product preferences and purchasing patterns to support predictions of future purchases.*

### **Model Training and Evaluation**

#### **Regression Approach using XGBoost and CatBoost**

**Model Training**

In [19]:
# Function to train a model
def train_model(X, y, model_type='catboost'):
    if model_type == 'catboost':
        model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=5, cat_features=['shopping_tendency'])
    elif model_type == 'xgboost':
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
    model.fit(X, y)
    return model

In [ ]:
X = train_df[['cluster', 'last_quantity','last_cumulative_quantity','shopping_tendency','cluster_customer',
              'product_popularity','days_since_first_purchase','rolling_avg_quantity','rolling_avg_days_between']]  # Features
y = train_df['log_days_since_last_purchase']  # Target

# Encode categorical column "shopping_tendency"
le_shopping_tendency = LabelEncoder()
X['shopping_tendency'] = le_shopping_tendency.fit_transform(X['shopping_tendency'])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\Aybike\AppData\Local\Temp\ipykernel_13052\1551150429.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['shopping_tendency'] = le_shopping_tendency.fit_transform(X['shopping_tendency'])


In [21]:
# Train CatBoost model
catboost_model = train_model(X_train, y_train, model_type='catboost')

0:	learn: 1.3876816	total: 205ms	remaining: 20.3s
1:	learn: 1.2544545	total: 289ms	remaining: 14.2s
2:	learn: 1.1349860	total: 387ms	remaining: 12.5s
3:	learn: 1.0280431	total: 474ms	remaining: 11.4s
4:	learn: 0.9325908	total: 559ms	remaining: 10.6s
5:	learn: 0.8467461	total: 660ms	remaining: 10.3s
6:	learn: 0.7701763	total: 748ms	remaining: 9.94s
7:	learn: 0.7018195	total: 835ms	remaining: 9.6s
8:	learn: 0.6411640	total: 924ms	remaining: 9.34s
9:	learn: 0.5874708	total: 1.02s	remaining: 9.21s
10:	learn: 0.5399205	total: 1.12s	remaining: 9.03s
11:	learn: 0.4979230	total: 1.21s	remaining: 8.86s
12:	learn: 0.4609181	total: 1.31s	remaining: 8.74s
13:	learn: 0.4286787	total: 1.39s	remaining: 8.56s
14:	learn: 0.4005836	total: 1.46s	remaining: 8.29s
15:	learn: 0.3761641	total: 1.53s	remaining: 8.04s
16:	learn: 0.3552426	total: 1.6s	remaining: 7.81s
17:	learn: 0.3371469	total: 1.67s	remaining: 7.6s
18:	learn: 0.3216545	total: 1.74s	remaining: 7.42s
19:	learn: 0.3083992	total: 1.81s	remaining:

In [22]:
# Train XGBoost model
xgboost_model = train_model(X_train, y_train, model_type='xgboost')

In [23]:
# Get predictions for the validation set in log scale
catboost_predictions_log = catboost_model.predict(X_val)
xgboost_predictions_log = xgboost_model.predict(X_val)

# Convert predictions and actual values back to original scale
catboost_predictions_original = np.exp(catboost_predictions_log)
xgboost_predictions_original = np.exp(xgboost_predictions_log)
y_val_original = np.exp(y_val)

# Calculate the mean squared error in the original scale
catboost_mse_original = mean_squared_error(y_val_original, catboost_predictions_original)
xgboost_mse_original = mean_squared_error(y_val_original, xgboost_predictions_original)

# Calculate the root mean squared error in the original scale
catboost_rmse_original = np.sqrt(catboost_mse_original)
xgboost_rmse_original = np.sqrt(xgboost_mse_original)

# Show the root mean squared error for each model
print("RMSE for CatBoost Model:", catboost_rmse_original)
print("RMSE for XGBoost Model:", xgboost_rmse_original)

RMSE for CatBoost Model: 15.650558096011206
RMSE for XGBoost Model: 15.508764583193665


**TEST SET EVALUATION**

In [24]:
# Function to make predictions and process results
def make_predictions(model, test, X_test, train_df_last, feb_1st_2021, map_weeks_to_feb,model_type):
    y_pred_log = model.predict(X_test)
    
    # Back-transform predictions
    test['prediction'] = np.expm1(y_pred_log)

    # Ensure the predicted days are integers
    test['prediction'] = test['prediction'].clip(lower=0).astype(int)

    # Merge the test data with the last purchase data to add the purchase date
    test_df_merged = test.merge(train_df_last[['customer_id', 'product_id', 'purchase_date']], 
                                  on=['customer_id', 'product_id'], how='left')

    # Initial predicted purchase date
    test_df_merged['predicted_purchase_date'] = test_df_merged['purchase_date'] + pd.to_timedelta(test_df_merged['prediction'], unit='D')

    # If the predicted date is before February 1, 2021, adjust the date
    def adjust_date(row):
        if(row['prediction']!=0):
            while row['predicted_purchase_date'] < pd.to_datetime('2021-02-01'):
                row['predicted_purchase_date'] += pd.to_timedelta(row['prediction'], unit='D')
            return row['predicted_purchase_date']

    test_df_merged['predicted_purchase_date'] = test_df_merged.apply(adjust_date, axis=1)

    # Calculate the number of days from February 1, 2021
    test_df_merged['days_from_feb_1st'] = (test_df_merged['predicted_purchase_date'] - feb_1st_2021).dt.days 

    # Apply the mapping function
    test_df_merged['prediction'] = test_df_merged['days_from_feb_1st'].apply(map_weeks_to_feb).astype(int)

    # Select relevant columns and save the result to a CSV file
    df_selected = test_df_merged[['id', 'customer_id', 'product_id', 'prediction']]
    print(f"Predictions of model{model_type}:")
    print(df_selected['prediction'].value_counts())
    df_selected.to_csv(f'predictions_{model_type}.csv', index=False)

In [25]:
# Map day values to target values - weeks
def map_weeks_to_feb(days):
    if days > 29:
        return 0
    elif 0 <= days <= 8:
        return 1
    elif 8 < days <= 15:
        return 2
    elif 15 < days <= 22:
        return 3
    elif 22 < days <= 29:
        return 4
    else:
        return 0

In [26]:
# Sort train_df by purchase_date in descending order for prediction
train_df['purchase_date'] = pd.to_datetime(train_df['purchase_date'])
train_df = train_df.sort_values(by='purchase_date', ascending=False)

# Create a new DataFrame containing only the last purchase for each customer-product pair
train_df_last = train_df.groupby(['customer_id', 'product_id']).first().reset_index()

# Read test csv
test = pd.read_csv("test.csv")

# Merge cluster and quantity information from the training DataFrame into the test DataFrame
test = pd.merge(test, train_df_last[['customer_id', 'product_id', 'cluster', 'quantity',
                                     'cumulative_quantity','shopping_tendency','cluster_customer',
                                     'product_popularity','days_since_first_purchase','rolling_avg_quantity',
                                     'rolling_avg_days_between']], 
                   on=['customer_id', 'product_id'], how='left')

# Rename the 'quantity' column to 'last_quantity'
test = test.rename(columns={'quantity': 'last_quantity','cumulative_quantity': 'last_cumulative_quantity'})

# Select relevant columns for testing
X_test = test[['cluster', 'last_quantity','last_cumulative_quantity','shopping_tendency',
               'cluster_customer','product_popularity','days_since_first_purchase','rolling_avg_quantity',
               'rolling_avg_days_between']]

C:\Users\Aybike\AppData\Local\Temp\ipykernel_13052\2919524360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['purchase_date'] = pd.to_datetime(train_df['purchase_date'])


In [27]:
feb_1st_2021 = pd.to_datetime('2021-02-01')

# Test CatBoost model
make_predictions(catboost_model, test, X_test, train_df_last, feb_1st_2021, map_weeks_to_feb,"catboost_model")


Predictions of modelcatboost_model:
prediction
0    5652
1    1193
4    1085
3    1039
2    1031
Name: count, dtype: int64


In [28]:
# Test XGBoost model
make_predictions(xgboost_model, test, X_test, train_df_last, feb_1st_2021, map_weeks_to_feb,"xgboost_model")

Predictions of modelxgboost_model:
prediction
0    5824
1    1131
4    1051
3    1042
2     952
Name: count, dtype: int64


#### **Classification Approach using XGBoost**

**Model Training**

Undersample Majority Class 

In [29]:
def undersample_and_train(X, y):
    
    df_train = pd.concat([X, y], axis=1)

    df_majority = df_train[df_train['target_weeks'] == 0]  # Majority class (0 weeks)
    df_minority = df_train[df_train['target_weeks'] != 0] 

    # Undersample the majority class to match the size of the minority class
    df_majority_undersampled = resample(df_majority, 
                                        replace=False,      # No replacement
                                        n_samples=len(df_minority),  # Match the minority class size
                                        random_state=42)

    # Combine the undersampled majority class with the minority class
    df_train_balanced = pd.concat([df_majority_undersampled, df_minority])

    # Shuffle the combined dataframe
    df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    # Separate the features and target again after balancing
    X_train_balanced = df_train_balanced.drop('target_weeks', axis=1)
    y_train_balanced = df_train_balanced['target_weeks']

    # Train an XGBoost model on the balanced training data
    model = XGBClassifier(objective='multi:softmax', num_class=5, n_estimators=100, learning_rate=0.1, max_depth=5)
    model.fit(X_train_balanced, y_train_balanced)
    
    return model


Oversample Minority Class

In [30]:
def oversample_and_train(X, y):
    df_train = pd.concat([X, y], axis=1)

    df_majority = df_train[df_train['target_weeks'] == 0]  # Majority class (0 weeks)
    df_minority = df_train[df_train['target_weeks'] != 0]  # Minority classes

    # Oversample the minority class to match the size of the majority class
    df_minority_oversampled = resample(df_minority, 
                                       replace=True,      # With replacement
                                       n_samples=len(df_majority),  # Match the majority class size
                                       random_state=42)

    # Combine the oversampled minority class with the majority class
    df_train_balanced = pd.concat([df_majority, df_minority_oversampled])

    # Shuffle the combined dataframe
    df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    # Separate the features and target again after balancing
    X_train_balanced = df_train_balanced.drop('target_weeks', axis=1)
    y_train_balanced = df_train_balanced['target_weeks']

    # Train an XGBoost model on the balanced training data
    model = XGBClassifier(objective='multi:softmax', num_class=5, n_estimators=100, learning_rate=0.1, max_depth=5)
    model.fit(X_train_balanced, y_train_balanced)
    
    return model

Same Sample Size For All Classes

In [31]:
def same_class_size_and_train(X, y):
    
    # Combine features and target into one DataFrame
    train_data = pd.concat([X, y], axis=1)

    min_sample_size = train_data['target_weeks'].value_counts().min()

    # Resample each class
    balanced_train_data = pd.concat([
        resample(train_data[train_data['target_weeks'] == cls],
                replace=False,  # No replacement
                n_samples=min_sample_size,  
                random_state=42)
        for cls in train_data['target_weeks'].unique()
    ])
    # Shuffle the balanced data
    balanced_train_data = balanced_train_data.sample(frac=1, random_state=42).reset_index(drop=True)

    # Separate features and target
    X_balanced = balanced_train_data.drop(columns=['target_weeks'])
    y_balanced = balanced_train_data['target_weeks']

    # Train an XGBoost Classifier
    model = XGBClassifier(objective='multi:softmax', num_class=5, n_estimators=100, learning_rate=0.1, max_depth=5)

    # Fit the model on the entire balanced dataset
    model.fit(X_balanced, y_balanced)
    
    return model


In [32]:
# Convert the 'days_since_last_purchase' into weeks and create target_weeks column
train_df['target_weeks'] = (train_df['days_since_last_purchase'] / 7).astype(int)

# Filter rows where the target_weeks is between 0 and 4 
train_df = train_df[(train_df['target_weeks'] >= 0) & (train_df['target_weeks'] <= 4)]

def map_weeks(days):
    if 0 < days <= 7:
        return 1
    elif 7 < days <= 14:
        return 2
    elif 14 < days <= 21:
        return 3
    elif 21 < days <= 28:
        return 4
    else:
        return 0

# Apply the mapping function to 'days_since_last_purchase' to create 'target_weeks'
train_df['target_weeks'] = train_df['days_since_last_purchase'].apply(map_weeks)
train_df['target_weeks'] .value_counts()

target_weeks
0    907570
4     10820
3      8122
2      5374
1      3641
Name: count, dtype: int64

In [33]:
# Encode the 'shopping_tendency' column as numbers
le_shopping_tendency = LabelEncoder()
train_df_encoded = train_df.copy()
train_df_encoded['shopping_tendency'] = le_shopping_tendency.fit_transform(train_df_encoded['shopping_tendency'])

X = train_df_encoded[['cluster', 'last_quantity','last_cumulative_quantity','shopping_tendency','cluster_customer',
              'product_popularity','days_since_first_purchase','rolling_avg_quantity','rolling_avg_days_between']]  # Features
y = train_df_encoded['target_weeks']  # Target

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [34]:
# Function to evaluate the model and print classification report
def evaluate_model(model, X_val, y_val):
    # Predict the validation set
    y_pred = model.predict(X_val)
    # Print classification report
    print(classification_report(y_val, y_pred, digits=2))  

In [35]:
# Train the XGBoost classifier on the split training data
xgboost_class = XGBClassifier(objective='multi:softmax', num_class=5, n_estimators=100, learning_rate=0.1, max_depth=5)
xgboost_class.fit(X_train, y_train)

print("Evaluation of XGBoost Classifier Trained on Split Training Data:")
evaluate_model(xgboost_class, X_val, y_val)

# Train the XGBoost classifier on the undersampled data
xgboost_class_undersample = undersample_and_train(X_train, y_train)

print("Evaluation of XGBoost Classifier Trained on Undersampled Data:")
evaluate_model(xgboost_class_undersample, X_val, y_val)


Evaluation of XGBoost Classifier Trained on Split Training Data:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    181515
           1       0.90      0.42      0.57       728
           2       0.64      0.31      0.42      1075
           3       0.55      0.28      0.37      1624
           4       0.43      0.52      0.47      2164

    accuracy                           0.98    187106
   macro avg       0.70      0.50      0.56    187106
weighted avg       0.98      0.98      0.98    187106

Evaluation of XGBoost Classifier Trained on Undersampled Data:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    181515
           1       0.95      0.47      0.63       728
           2       0.68      0.31      0.43      1075
           3       0.42      0.32      0.36      1624
           4       0.35      0.92      0.51      2164

    accuracy                           0.98    187106
   

In [36]:
# Train the XGBoost classifier on the oversampled data
xgboost_class_oversample = oversample_and_train(X_train, y_train)

print("Evaluation of XGBoost Classifier Trained on Oversampled Data:")
evaluate_model(xgboost_class_oversample, X_val, y_val)

# Train the XGBoost classifier on the data that have same number of samples from each class
xgboost_same_size_class = same_class_size_and_train(X_train, y_train)

print("Evaluation of XGBoost Classifier Trained on Balanced Data:")
evaluate_model(xgboost_same_size_class, X_val, y_val)

Evaluation of XGBoost Classifier Trained on Oversampled Data:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    181515
           1       0.95      0.47      0.63       728
           2       0.69      0.31      0.43      1075
           3       0.42      0.32      0.37      1624
           4       0.35      0.92      0.51      2164

    accuracy                           0.98    187106
   macro avg       0.68      0.60      0.58    187106
weighted avg       0.99      0.98      0.98    187106

Evaluation of XGBoost Classifier Trained on Balanced Data:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    181515
           1       0.63      0.50      0.55       728
           2       0.42      0.39      0.41      1075
           3       0.34      0.40      0.37      1624
           4       0.37      0.75      0.50      2164

    accuracy                           0.97    187106
   macro a

Evaluation of Classification Models on Test Set

In [37]:
# Sort train_df by purchase_date in descending order for prediction
train_df['purchase_date'] = pd.to_datetime(train_df['purchase_date'])
train_df = train_df.sort_values(by='purchase_date', ascending=False)

# Create a new DataFrame containing only the last purchase for each customer-product pair
train_df_last = train_df.groupby(['customer_id', 'product_id']).first().reset_index()

# Read test csv
test = pd.read_csv("test.csv")

# Merge cluster and quantity information from the training DataFrame into the test DataFrame
test = pd.merge(test, train_df_last[['customer_id', 'product_id', 'cluster', 'quantity',
                                     'cumulative_quantity','shopping_tendency','cluster_customer',
                                     'product_popularity','days_since_first_purchase','rolling_avg_quantity',
                                     'rolling_avg_days_between']], 
                   on=['customer_id', 'product_id'], how='left')

# Rename the 'quantity' column to 'last_quantity'
test = test.rename(columns={'quantity': 'last_quantity','cumulative_quantity': 'last_cumulative_quantity'})

# Select relevant columns for testing
X_test = test[['cluster', 'last_quantity','last_cumulative_quantity','shopping_tendency',
               'cluster_customer','product_popularity','days_since_first_purchase','rolling_avg_quantity',
               'rolling_avg_days_between']]

In [38]:
# Function to evaluate the model on test data and write predictions to a CSV file
def evaluate_and_write_to_csv(test, X_test, model, name):
    test['prediction'] = model.predict(X_test)  # Predict test set
    print(test['prediction'].value_counts())
    df_selected = test[['id', 'customer_id', 'product_id', 'prediction']]
    df_selected.to_csv(f'predictions_{name}.csv', index=False)  # Save as CSV

In [39]:
evaluate_and_write_to_csv(test, X_test, xgboost_class, "xgboost_class")
evaluate_and_write_to_csv(test, X_test, xgboost_class_undersample, "xgboost_class_undersample")
evaluate_and_write_to_csv(test, X_test, xgboost_class_oversample, "xgboost_class_oversample")
evaluate_and_write_to_csv(test, X_test, xgboost_same_size_class, "xgboost_sames_size_class")

prediction
0    5849
4    1987
1    1602
3     374
2     188
Name: count, dtype: int64
prediction
4    5477
0    3653
3     597
2     179
1      94
Name: count, dtype: int64
prediction
4    5471
0    3675
3     609
2     167
1      78
Name: count, dtype: int64
prediction
4    4228
1    3905
3    1371
2     496
Name: count, dtype: int64


2-Stage Prediction Model

1: Predict if customer will buy in 4 weeks or not

2: Predict in which week will buy the product if s/he will buy

In [40]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert y to binary for the first model (0 for no purchase, 1 for purchase)
y_train_binary = (y_train != 0).astype(int)  # Class 0 stays 0, all others become 1
y_val_binary = (y_val != 0).astype(int)

# Train the binary classifier to predict if a customer will buy or not
binary_model = XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, max_depth=5)
binary_model.fit(X_train, y_train_binary)

# Make predictions with the binary model on the validation set
binary_predictions_val = binary_model.predict(X_val)

# Identify validation samples predicted to buy 
val_indices_non_0 = binary_predictions_val == 1  # Customers predicted to buy
X_val_non_0 = X_val[val_indices_non_0]
y_val_non_0 = y_val[val_indices_non_0] - 1  # Map classes to [0, 1, 2, 3]

# Train the multi-class model to predict which week the customer will buy
train_indices_non_0 = y_train_binary == 1  
X_train_non_0 = X_train[train_indices_non_0]
y_train_non_0 = y_train[train_indices_non_0] - 1  # Map classes to [0, 1, 2, 3]

multi_class_model = XGBClassifier(objective='multi:softmax', num_class=4, n_estimators=100, learning_rate=0.1, max_depth=5)
multi_class_model.fit(X_train_non_0, y_train_non_0)

# Make predictions with the multi-class model for customers predicted to buy
multi_class_predictions_val = multi_class_model.predict(X_val_non_0)

# Map multi-class predictions back to original weeks (1, 2, 3, 4)
multi_class_predictions_val = multi_class_predictions_val + 1

# Create a combined prediction array for the validation set
combined_predictions_val = binary_predictions_val.copy()
combined_predictions_val[val_indices_non_0] = multi_class_predictions_val

# Print the classification report for the validation set
print("Evaluation of 2-stage model on validation set:")
print(classification_report(y_val, combined_predictions_val))

# Make predictions with the binary model on the test data
binary_predictions_test = binary_model.predict(X_test)

# Identify non-0 predictions and predict which week they will buy using the multi-class model
test_indices_non_0 = binary_predictions_test == 1  # Samples predicted as 1 by the binary model
X_test_non_0 = X_test[test_indices_non_0]

# Predict the week of purchase for customers who are predicted to buy
multi_class_predictions_test = multi_class_model.predict(X_test_non_0)

# Map predictions back to the original weeks (1, 2, 3, 4)
multi_class_predictions_test = multi_class_predictions_test + 1

# Add the binary predictions and multi-class predictions to the test data
test['prediction'] = binary_predictions_test
test.loc[test_indices_non_0, 'prediction'] = multi_class_predictions_test  # Replace with multi-class predictions for those who will buy

# Save the final predictions to a CSV file
test[['id', 'customer_id', 'product_id', 'prediction']].to_csv('2-stage-model-predictions.csv', index=False)



Evaluation of 2-staged model on validation set:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    181556
           1       0.89      0.43      0.58       694
           2       0.72      0.36      0.48      1074
           3       0.59      0.34      0.44      1584
           4       0.37      0.92      0.53      2198

    accuracy                           0.98    187106
   macro avg       0.72      0.61      0.61    187106
weighted avg       0.99      0.98      0.98    187106



In [41]:
test['prediction'].value_counts()

prediction
4    5362
0    2267
1    1725
3     480
2     166
Name: count, dtype: int64